# Cinématique inverse

Ce notebook explore la génération de mouvements pour un robot humanoïde en utilisant la **cinématique inverse**.  
L'exercice consiste à générer une trajectoire où :
- Le **centre de masse (CoM)** du robot suit une trajectoire à vitesse constante.
- Les **pieds** (gauche et droit) effectuent des **déplacements alternés**, comme lors d'une marche, également à vitesse constante.

Nous utiliserons pour cela une approche **cinématique**, sans résolution dynamique, et enchaînerons des tâches résolues via pseudo-inversion du jacobien (commande à vitesse).

Le mouvement sera composé de plusieurs **phases**, alternant appui gauche et appui droit.

In [1]:
import gepetuto

### Chargement du robot et initialisation du visualiseur

Entête classique avec Pinocchio, example robot data et meshcat

In [ ]:
%load tp4/gener